In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report


In [4]:
# Carregar os dados
df = pd.read_csv('INFLUD23.csv', sep=';', encoding='latin1')

C:\Users\Mateu\AppData\Local\Temp\ipykernel_11624\2030035574.py:2: DtypeWarning: Columns (15,18,20,23,61,62,63,72,74,79,91,93,108,109,120,130,143,171,175,178) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('INFLUD23.csv', sep=';', encoding='latin1')


In [4]:
# 2. Criar variáveis compostas básicas
df['SINTOMA_GRAVE'] = ((df['DISPNEIA'] == 1) | (df['DESC_RESP'] == 1) | (df['SATURACAO'] == 1)).astype(int)
df['COMORBIDADE_GRAVE'] = ((df['DIABETES'] == 1) | (df['CARDIOPATI'] == 1) | (df['OBESIDADE'] == 1)).astype(int)
df['INTERNACAO_GRAVE'] = ((df['HOSPITAL'] == 1) | (df['UTI'] == 1)).astype(int)

# 3. Definir as variáveis utilizadas no modelo
variaveis_modelo = [
    # Sintomas
    'FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO',
    'DIARREIA', 'VOMITO', 'OUTRO_SIN', 'DOR_ABD', 'FADIGA', 'PERD_OLFT', 'PERD_PALA',

    # Comorbidades
    'PUERPERA', 'FATOR_RISC', 'CARDIOPATI', 'HEMATOLOGI', 'HEPATICA', 'ASMA',
    'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE', 'OBES_IMC',

    # Gravidade
    'HOSPITAL', 'UTI', 'SUPORT_VEN', 'RAIOX_RES', 'TOMO_RES',

    # Vacinação
    'VACINA_COV', 'DOSE_1_COV', 'DOSE_2_COV',

    # Variáveis compostas
    'SINTOMA_GRAVE', 'COMORBIDADE_GRAVE', 'INTERNACAO_GRAVE'
]

In [5]:
# 4. Definir o alvo
y_total = df['CLASSI_FIN']
X = df[variaveis_modelo]

# 5. Pré-processamento
X = X.replace(9, np.nan)
X = X.dropna(thresh=len(variaveis_modelo) - 5)
X = X.fillna(0)
y = y_total.loc[X.index]

validos = ~y.isna()
X = X.loc[validos]
y = y.loc[validos]

X = X[y != 4]
y = y[y != 4]

# 6. Corrigir tipos de variáveis
colunas_para_numeric = ['OBES_IMC', 'DOSE_1_COV', 'DOSE_2_COV']
for col in colunas_para_numeric:
    X[col] = pd.to_numeric(X[col], errors='coerce')

# Mapear resultado de exames de imagem
mapeamento_resultado_imagem = {
    'NORMAL': 0,
    'ALTERADO': 1,
    'INDETERMINADO': 2
}
X['RAIOX_RES'] = X['RAIOX_RES'].map(mapeamento_resultado_imagem)
X['TOMO_RES'] = X['TOMO_RES'].map(mapeamento_resultado_imagem)

# 7. Separação treino/teste (SEM STRATIFY)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

<ipython-input-5-5edf317a56dd>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = pd.to_numeric(X[col], errors='coerce')
<ipython-input-5-5edf317a56dd>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['RAIOX_RES'] = X['RAIOX_RES'].map(mapeamento_resultado_imagem)
<ipython-input-5-5edf317a56dd>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

In [6]:
# 8. Mapear classes
mapeamento_classes = {1: 0, 2: 1, 3: 2, 5: 3}
y_train = y_train.map(mapeamento_classes)
y_test = y_test.map(mapeamento_classes)

# 9. Definir o modelo base
modelo_base = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

In [7]:
# 10. Definir o grid de hiperparâmetros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1]
}

# 11. Configurar o GridSearchCV
grid_search = GridSearchCV(estimator=modelo_base, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2)

# 12. Treinar o GridSearch
grid_search.fit(X_train, y_train)

# 13. Avaliar o melhor modelo
melhor_modelo = grid_search.best_estimator_

print("\nMelhores parâmetros:", grid_search.best_params_)
print("Acurácia do melhor modelo:", melhor_modelo.score(X_test, y_test))

Fitting 3 folds for each of 36 candidates, totalling 108 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:39:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Melhores parâmetros: {'learning_rate': 0.2, 'max_depth': 4, 'n_estimators': 100, 'subsample': 1}
Acurácia do melhor modelo: 0.6315563889633002


In [8]:
print(df.shape)
print(df.columns.tolist())
print(df['CLASSI_FIN'].value_counts(dropna=False))

(547484, 193)
['DT_NOTIFIC', 'SEM_NOT', 'DT_SIN_PRI', 'SEM_PRI', 'SG_UF_NOT', 'ID_REGIONA', 'CO_REGIONA', 'ID_MUNICIP', 'CO_MUN_NOT', 'ID_UNIDADE', 'CO_UNI_NOT', 'CS_SEXO', 'DT_NASC', 'NU_IDADE_N', 'TP_IDADE', 'COD_IDADE', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'ID_PAIS', 'CO_PAIS', 'SG_UF', 'ID_RG_RESI', 'CO_RG_RESI', 'ID_MN_RESI', 'CO_MUN_RES', 'CS_ZONA', 'SURTO_SG', 'NOSOCOMIAL', 'AVE_SUINO', 'FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO', 'OUTRO_SIN', 'OUTRO_DES', 'PUERPERA', 'FATOR_RISC', 'CARDIOPATI', 'HEMATOLOGI', 'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE', 'OBES_IMC', 'OUT_MORBI', 'MORB_DESC', 'VACINA', 'DT_UT_DOSE', 'MAE_VAC', 'DT_VAC_MAE', 'M_AMAMENTA', 'DT_DOSEUNI', 'DT_1_DOSE', 'DT_2_DOSE', 'ANTIVIRAL', 'TP_ANTIVIR', 'OUT_ANTIV', 'DT_ANTIVIR', 'HOSPITAL', 'DT_INTERNA', 'SG_UF_INTE', 'ID_RG_INTE', 'CO_RG_INTE', 'ID_MN_INTE', 'CO_MU_INTE', 'UTI', 'DT_ENTUTI', 'DT_SAIDUTI

In [ ]:
pip install xgboost